# Prototyping Dialogue Games
### Disclaimer
This notebook closely follows the tutorial steps from **docs/howto_prototype_games.ipynb** in the [**clp-research/clembench**](https://github.com/clp-research/clembench) repository. <br> 
While the core content is derived from the original tutorial, some additional detailed descriptions and explanations have been added to serve as a source of knowledge during the development of our project.
### Step 1: Necessary Imports

In [1]:
import sys
from string import Template

# Setting the correct directory so that import statements do not fail
# If you are unsure about the directory, just type pwd in your terminal
sys.path.append('/home/zoltan/Desktop/Gruppe-C-PRO2-Projekt/clembench-main')

from backends import Model, get_model_for, load_model_registry

### Step 2: Specifying the Model

In [2]:
# Load the default model registry from the backends folder
# You can inspect the registry when opening backends/model_registry.json
load_model_registry()

# Specify the model
THIS_MODEL = 'gpt-4o-mini-2024-07-18'
llm: Model = get_model_for(THIS_MODEL)

# Set generation arguments
# temperature: the lower it is, the more deterministic the output becomes
# max_tokens: the amount of tokens/words the model is allowed to use in a response
llm.set_gen_args(temperature=0.0, max_tokens=100)

### Step 3: Adding Messages
This function appends a new message to the `context`, which is a list of dictionaries where each dictionary represents a message in the conversation. The `sysmsg` is called if no conversation has been initialized yet.

In [3]:
def add_message(context, msg, role='user', sysmsg = 'You are a player in a game'):
    if context == []:
        context = [
            {"role": "system", "content": sysmsg}
        ]
    context.append({"role": role, "content": msg})
    return context

In [4]:
# Adding a message, we can see the resulting lis of dictionaries
add_message([], "How old is the sun")

[{'role': 'system', 'content': 'You are a player in a game'},
 {'role': 'user', 'content': 'How old is the sun'}]

In [5]:
prompt = add_message([], "How old is the sun?")
# resp_text = llm.generate_response(prompt)   # Returns the entire tuple from generate_response
prompt, resp, resp_text = llm.generate_response(prompt) # Returns only the contents of the generated result
print(resp_text)

The Sun is approximately 4.6 billion years old. This age is estimated based on the study of solar system formation and the ages of the oldest meteorites.


Here is an example of the complete structure of the returned tuple, where it is easier to see each detail:
```json
[
  {
    "content": [
      {
        "text": "You are a player in a game",
        "type": "text"
      }
    ],
    "role": "system"
  },
  {
    "content": [
      {
        "text": "How old is the sun?",
        "type": "text"
      }
    ],
    "role": "user"
  }
],
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The Sun is approximately 4.6 billion years old. This age is estimated based on the study of solar system formation and the ages of the oldest meteorites.",
        "function_call": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": null
      }
    }
  ],
  "created": 1724755685,
  "id": "chatcmpl-A0o21VxrNbI1bSALI1bpUA1XFE3qG",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_507c9469a1",
  "usage": {
    "completion_tokens": 33,
    "prompt_tokens": 24,
    "total_tokens": 57
  }
},
"The Sun is approximately 4.6 billion years old. This age is estimated based on the study of solar system formation and the ages of the oldest meteorites."


# Prototyping a game
There are two important elements that need to be considered:
##### MOVE_RULE
this somewhat defines, what a valid move (what form it takes) is

##### GAME_RULE
This contains rules that determine what effect a valid move from above should have on the game state, whether a move is legal, whether it updates the game state to `finished`.

### Example Game
Player A and B take turns, while they have a conversation. Every first word has to start with the next letter of the alphabet, based on the first word of the previous turn and so on, while sticking to a specific topic.


### Player A:

In [6]:
init_prompt_A = Template('''Let us play a game. I will tell you a topic and you will give me a short sentence that fits to this topic.
But I will also tell you a letter. The sentence that you give me has to start with this letter.
After you have answered, I will give you my reply, which will start with the letter following your letter in the alphabet.
Then it is your turn again, to produce a reply starting with the letter following that one. And so on. Let's go.
Start your utterance with I SAY: and do no produce any other text.
The topic is: $topic
The letter is: $letter''')

In [7]:
topic = 'clothes'
letter = 'b'
# Substitute here goes into the init_prompt_A template and replaces $topic and $letter with the variables defined above
# This will be logic every single time we call this below
init_prompt_A.substitute(topic=topic, letter=letter)

"Let us play a game. I will tell you a topic and you will give me a short sentence that fits to this topic.\nBut I will also tell you a letter. The sentence that you give me has to start with this letter.\nAfter you have answered, I will give you my reply, which will start with the letter following your letter in the alphabet.\nThen it is your turn again, to produce a reply starting with the letter following that one. And so on. Let's go.\nStart your utterance with I SAY: and do no produce any other text.\nThe topic is: clothes\nThe letter is: b"

### What Happens here?
So now the initial prompt is defined, along with the topic and the starting letter. <br> Understanding what is happening here is easier, if you think about this part as the start of a conversation with a language model like **ChatGPT**. <br> `init_prompt_A` would basically be your first message to ChatGPT. See: <br> <br> 

<img src="../../utils/gpt_first_prompt.png" width="1200" style="border: 2px solid black;" />

<br clear="both"/>

Our goal with the game is that we implement a GameMaster that includes `GAME_RULE` and `MOVE_RULE`, which allow us to actually control the flow of the game <br>
We could not do that, if we were to just continue the game with ChatGPT in the picture above. 


In [8]:
prompt_A, resp, resp_text = llm.generate_response(add_message([], init_prompt_A.substitute(topic='clothes', letter='b')))
resp_text

'I SAY: Blue jeans are a timeless wardrobe staple.'

### Adding MOVE_RULE

In [9]:
# As we defined in the initial prompt, what our LM returns, has to start with the prefix 'I SAY'
# If it is not the case, the function returns False = Wrong guess (?)
prefix = 'I SAY: '
def parse_reply(text, prefix=prefix):
    if not text.startswith(prefix):
        return False
    return text[len(prefix):]
    # The text is returned, but without the prefix (the length of the prefix is 'cut' from the string)

Let's see what `parse_reply` returns. <br>
If we have a sentence without `'I SAY'`, that means, the prompt worked and the LM produced a valid guess.
<br> If we see **False**, then the guess is incorrect, or the LM did not understand the prompt.

In [10]:
parse_reply(resp_text)

'Blue jeans are a timeless wardrobe staple.'

### Adding GAME_RULE

In [11]:
# This function checks, whether the first letter of the first word in the tokenized text is the same as the 'letter' argument
def check_move(text, letter):
    token = text.split()
    if token[0][0].lower() == letter: # and token[-1][0].lower() == letter:
        return True
    return False

In [12]:
check_move(parse_reply(resp_text), letter)

True

### Player B:
This part is pretty self-explanatory, the prompt is a bit different, than what is given to Player A, as there was an initial turn already. <br> 
That means we already have a sentence that has to be passed on to player B as part of this prompt:

In [13]:
init_prompt_B = Template('''Let us play a game. I will give you a sentence.
The first word in my sentence starts with a certain letter.
I want you to give me a sentence as a reply, with the same topic as my sentence, but different from my sentence.
The first word of your sentence should start with the next letter in the alphabet from the letter my sentence started with.
Let us try to have a whole conversation like that.
Please start your reply with 'I SAY:' and do not produce any other text.
Let's go.
My sentence is: $sentence
What do you say?''')

In [14]:
sentence = parse_reply(resp_text) # once again, the first response is validated and passed on as the argument 'sentence'
prompt_B = init_prompt_B.substitute(sentence=sentence)
prompt_B

"Let us play a game. I will give you a sentence.\nThe first word in my sentence starts with a certain letter.\nI want you to give me a sentence as a reply, with the same topic as my sentence, but different from my sentence.\nThe first word of your sentence should start with the next letter in the alphabet from the letter my sentence started with.\nLet us try to have a whole conversation like that.\nPlease start your reply with 'I SAY:' and do not produce any other text.\nLet's go.\nMy sentence is: Blue jeans are a timeless wardrobe staple.\nWhat do you say?"

In [15]:
# Now an actual response is generated. The functionality here is the same as for Player A
prompt_B, resp_B, resp_B_text = llm.generate_response(add_message([], prompt_B))
resp_B_text

'I SAY: Classic trousers can elevate any outfit effortlessly.'

In [16]:
# Response is validated and the prefix 'I SAY' gets ommited.
parse_reply(resp_B_text)

'Classic trousers can elevate any outfit effortlessly.'

# Handling Game States
Now that the initial prompts have been defined, the state of the game has to be handled. We need to validate the Move. If it is correct, the game state is updated, until a finishing level is met. If the move does not valide, the game is aborted.

In [17]:
class InitialsGameState():
    def __init__(self, letter):
        self.letter = letter
        self.n_moves = 0
        self.success = False
        self.aborted = False
    
    # Update the letter after a successful turn
    def increment_state(self):
        self.letter = chr(ord(self.letter) + 1 )
        self.n_moves += 1

In [36]:
this_game = InitialsGameState('b') # Create an Instance of the GameState Class
# this_game.increment_state() # Updating to next letter in the alphabet
this_game.letter, this_game.n_moves

('b', 0)

In [19]:
# Check if the first letter of the parsed response matches the class attribute letter
check_move(parse_reply(resp_B_text), this_game.letter)

True

# The Game Loop
From here on, the game consists of giving A the turn, parsing and validating the response, then the same with B. The game has to be stopped, if: <br>
- Unparseable move detected (not understandable response) <br>
- A losing move was made (wrong initial) <br>
- Maximum number of turns reached


In [20]:
next_prompt_A = add_message(prompt_A, resp_text, role='assistant')
next_prompt_A = add_message(next_prompt_A, resp_B_text, role='user')
next_prompt_A

[{'role': 'system',
  'content': [{'type': 'text', 'text': 'You are a player in a game'}]},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': "Let us play a game. I will tell you a topic and you will give me a short sentence that fits to this topic.\nBut I will also tell you a letter. The sentence that you give me has to start with this letter.\nAfter you have answered, I will give you my reply, which will start with the letter following your letter in the alphabet.\nThen it is your turn again, to produce a reply starting with the letter following that one. And so on. Let's go.\nStart your utterance with I SAY: and do no produce any other text.\nThe topic is: clothes\nThe letter is: b"}]},
 {'role': 'assistant',
  'content': 'I SAY: Blue jeans are a timeless wardrobe staple.'},
 {'role': 'user',
  'content': 'I SAY: Classic trousers can elevate any outfit effortlessly.'}]

In [28]:
resp_text

'I SAY: Blue jeans are a timeless wardrobe staple.'

In [38]:
psd_reply = parse_reply(resp_text)
if psd_reply:
    if check_move(psd_reply, letter=this_game.letter):
        print('YAY')
    else:
        print('LOST')
else:
    print('NOT WELL-FORMED')

YAY


In [41]:
next_prompt_B = add_message(prompt_B, resp_B_text, role='assistant')
next_prompt_B = add_message(next_prompt_B, resp_text, role='user')
next_prompt_B

[{'role': 'system',
  'content': [{'type': 'text', 'text': 'You are a player in a game'}]},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': "Let us play a game. I will give you a sentence.\nThe first word in my sentence starts with a certain letter.\nI want you to give me a sentence as a reply, with the same topic as my sentence, but different from my sentence.\nThe first word of your sentence should start with the next letter in the alphabet from the letter my sentence started with.\nLet us try to have a whole conversation like that.\nPlease start your reply with 'I SAY:' and do not produce any other text.\nLet's go.\nMy sentence is: Blue jeans are a timeless wardrobe staple.\nWhat do you say?"}]},
 {'role': 'assistant',
  'content': 'I SAY: Classic trousers can elevate any outfit effortlessly.'},
 {'role': 'user',
  'content': 'I SAY: Blue jeans are a timeless wardrobe staple.'},
 {'role': 'assistant',
  'content': 'I SAY: Classic trousers can elevate any outfit effortle

In [43]:
prompt_B, resp_B, resp_B_text = llm.generate_response(next_prompt_B)
resp_B_text

2024-08-28 17:31:42,858 - backends.openai_api - WARNING - 'list' object has no attribute 'replace', retrying in 0 seconds...
2024-08-28 17:31:42,864 - backends.openai_api - WARNING - 'list' object has no attribute 'replace', retrying in 0 seconds...


AttributeError: 'list' object has no attribute 'replace'

In [ ]:
this_game.increment_state()

In [45]:
psd_reply = parse_reply(resp_B_text)
if psd_reply:
    if check_move(psd_reply, letter=this_game.letter):
        print('YAY')
    else:
        print('LOST')
else:
    print('NOT WELL-FORMED')

Classic trousers can elevate any outfit effortlessly.
b
LOST
